In [ ]:
import numpy as np

In [ ]:
def activation_function(x, function='sigmoid'):
    if function  == 'sigmoid':
        x = 1 / (1 + np.exp(-x))
    elif function == 'step':
        x = 1 if x >= 0 else 0
    elif function == 'softmax':
        x = np.exp(x)
        x = np.divide (x, x.sum())
    return x
        

In [ ]:
def calc_score(X, W, b):
    score = np.matmul(X,W) + b
    return score


In [ ]:
def perceptronStep(X, y, W, b, learn_rate = 0.01):
    for i in range(len(X)):
        if calc_score(X[i], W, b) != y[i]:
            if calc_score(X[i], W, b)  == 0 :
                for j in range(len(X[i])):
                    W[j] = W[j] + learn_rate*X[i][j]
                b = b + learn_rate
            else:
                for j in range(len(X[i])):
                    W[j] = W[j] - learn_rate*X[i][j]
                b = b - learn_rate
    return W, b

In [ ]:
def error(y, p, formula = 'cross_entropy'):
    if formula == 'cross_entropy':
        y = np.float_(y)
        p = np.float_(p)
        error = -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return error

In [ ]:
def training(X, y, learn_rate = 0.01, num_epochs = 10):
    for i in range(num_epochs):
        W, b = perceptronStep(X, y, W, b, learn_rate)
        history = []
        history.append(W, b)
    return history

In [ ]:
print (history)

In [ ]:
#example
history = []
X = ([][][])